### Сборный проект №1
[ссылка на практикум](https://practicum.yandex.ru/learn/data-scientist/courses/05fc6f4e-775b-43b8-b9d3-6eb470e1a00a/sprints/176021/topics/2bb8744d-a0e9-4da9-9cfc-8a23a90a39b1/lessons/48a04d31-755b-4ca1-a953-1a9bd1b3169e/)

#### Описание проекта


Вы работаете в интернет-магазине «Стримчик», который продаёт по всему миру компьютерные игры. Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation). Вам нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

Перед вами данные до 2016 года. Представим, что сейчас декабрь 2016 г., и вы планируете кампанию на 2017-й. Нужно отработать принцип работы с данными. Неважно, прогнозируете ли вы продажи на 2017 год по данным 2016-го или же 2027-й — по данным 2026 года.

В наборе данных попадается аббревиатура ESRB (Entertainment Software Rating Board) — это ассоциация, определяющая возрастной рейтинг компьютерных игр. ESRB оценивает игровой контент и присваивает ему подходящую возрастную категорию, например, «Для взрослых», «Для детей младшего возраста» или «Для подростков».

#### Шаг 1. Открыть данные, изучить информацию

In [1]:
import pandas as pd

In [ ]:
# раскомментировать строку с путем до файла для ревьюера
"""
dataset_link = '/datasets/games.csv'

"""

In [2]:
# закомментировать строку с путем до файла( на моем диске)

dataset_link = '/Users/tony_builder/Documents/GitHub/studying/yandex_practicum/data/games.csv'


In [3]:
data = pd.read_csv(dataset_link)
data.head()

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [4]:
data.shape

(16715, 11)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


<font color='red'>**Вывод по шагу 1:**</font>

- 16715 строк, 11 столбцов
- есть пустые значения 
- названия столбцов нужно привести в соответствие

<font color='green'>**Итог:**</font> данные требуют предобработки

#### Шаг 2. Подготовка данных для анализа

1) Сначала приведем все имена столбцов к нижнему регистру

In [8]:
for col in data.columns:
    data = data.rename(columns={col: col.lower()})

In [10]:
data.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

Выше видим, что все сработало. Но тут мало столбцов, а так можно проверить, что все столбцы теперь в нижнем регистре: 

- возьмем итератор, в котором заменим все названия столбца на булево значение (проверка на то, что название столбца в нижнем регистре)
- посчитаем сумму итератора (должны быть все единички)
- вычтем число столбцов = 11

Должно получиться 0

In [15]:
sum((col.islower() for col in data.columns)) - 11

0

2) Теперь поработаем с пропусками

2.1 в столбце name 

In [16]:
data[data.name.isna()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14244,NaN,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


всего два значения, причем второе не вызывает никаких сомнений - удалить без жалости.

а что со вторым? по нему могут быть интересны только продажи по году (1993) и по платформе (GEN) - именно на эти параметры могут повлиять суммы продаж при построении сводных. Посмотрим на них внимательно


In [18]:
data.groupby('year_of_release').\
    agg({'na_sales': 'sum', 'eu_sales': 'sum', 'other_sales': 'sum'}).\
    query('year_of_release == 1993.0')

,na_sales,eu_sales,other_sales
year_of_release,,,
1993.0,16.9,5.18,0.97


ну вообще-то продажи безымянной игры по регионам составляют аж 10% от общих продаж по году.

пренебрегать такими данными нельзя.

In [19]:
# убираем только вторую строку
data.drop(index=14244, inplace=True)

In [20]:
data[data.name.isna()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993.0,NaN,1.78,0.53,0.0,0.08,NaN,NaN,NaN


Неплохо бы поменять тип столбца год выпуска на целочисленное значение, но там 269 пустых значений с типом float.

посмотрим что это значения

2.2 пропуски в столбце год выпуска

In [24]:
data[data.year_of_release.isna()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
183,Madden NFL 2004,PS2,NaN,Sports,4.26,0.26,0.01,0.71,94.0,8.5,E
377,FIFA Soccer 2004,PS2,NaN,Sports,0.59,2.36,0.04,0.51,84.0,6.4,E
456,LEGO Batman: The Videogame,Wii,NaN,Action,1.80,0.97,0.00,0.29,74.0,7.9,E10+
475,wwe Smackdown vs. Raw 2006,PS2,NaN,Fighting,1.57,1.02,0.00,0.41,NaN,NaN,NaN
609,Space Invaders,2600,NaN,Shooter,2.36,0.14,0.00,0.03,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16373,PDC World Championship Darts 2008,PSP,NaN,Sports,0.01,0.00,0.00,0.00,43.0,tbd,E10+
16405,Freaky Flyers,GC,NaN,Racing,0.01,0.00,0.00,0.00,69.0,6.5,T
16448,Inversion,PC,NaN,Shooter,0.01,0.00,0.00,0.00,59.0,6.7,M
16458,Hakuouki: Shinsengumi Kitan,PS3,NaN,Adventure,0.01,0.00,0.00,0.00,NaN,NaN,NaN


Интересно: в названии игр встречается упоминание о годе. Может отсюда можно почерпнуть информацию для пустых значений по году?

Проверим это:
- сделаем срез по всем играм, у которых указан год в имени
- оставим только те, у которых значение в столбце год выпуска не пустое
- если они совпадают  - можно сделать вывод, что это применимо и к остальным играм

In [40]:
import re
import numpy as np

In [37]:
regex = r"[0-9]{4}"

print(re.search(regex, 'FIFA Soccer 2004').group())

2004


In [61]:
def find_year_in_name(string):
    regex = r"[0-9]{4}"
    match = re.search(regex, str(string))
    return match.group() if match else None

In [63]:
data['year_in_name'] = data.name.apply(find_year_in_name)

In [64]:
data[~data.year_in_name.isna()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,year_in_name
87,Star Wars Battlefront (2015),PS4,2015.0,Shooter,2.99,3.49,0.22,1.28,NaN,NaN,NaN,2015
183,Madden NFL 2004,PS2,NaN,Sports,4.26,0.26,0.01,0.71,94.0,8.5,E,2004
206,Warzone 2100,PS,1999.0,Strategy,2.79,1.89,0.00,0.33,NaN,NaN,NaN,2100
238,Madden NFL 2005,PS2,2004.0,Sports,4.18,0.26,0.01,0.08,91.0,7.9,E,2005
250,Winning Eleven: Pro Evolution Soccer 2007,PS2,2006.0,Sports,0.10,2.39,1.05,0.86,86.0,8.9,E,2007
...,...,...,...,...,...,...,...,...,...,...,...,...
16676,G1 Jockey 4 2008,PS3,2008.0,Sports,0.00,0.00,0.00,0.00,NaN,NaN,NaN,2008
16697,Bust-A-Move 3000,GC,2003.0,Puzzle,0.01,0.00,0.00,0.00,53.0,tbd,E,3000
16709,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,0.00,0.00,0.00,0.00,NaN,NaN,NaN,1000
16711,LMA Manager 2007,X360,2006.0,Sports,0.00,0.01,0.00,0.00,NaN,NaN,NaN,2007


Бегло пройдясь сходу видны и явные ошибки такого метода - например, в названии игры "SCORE International Baja 1000: The Official Game" есть цифра 1000, но разумеется это не может быть годом.

вывод - нужно немного доработать функцию find_year_in_name

In [66]:
# посмотрим весь дипазон годов выпуска игр
sorted(data.year_of_release.unique())

[1980.0,
 1981.0,
 1982.0,
 1983.0,
 1984.0,
 1985.0,
 1986.0,
 1987.0,
 1988.0,
 1989.0,
 1990.0,
 1991.0,
 1992.0,
 1993.0,
 1994.0,
 1995.0,
 1996.0,
 1997.0,
 1998.0,
 1999.0,
 2000.0,
 2001.0,
 2002.0,
 2003.0,
 2004.0,
 2005.0,
 2006.0,
 2007.0,
 2008.0,
 2009.0,
 2010.0,
 2011.0,
 2012.0,
 2013.0,
 2014.0,
 2015.0,
 2016.0,
 nan]

In [74]:
def find_year_in_name_2(string):
    regex = r"[0-9]{4}"
    match = re.search(regex, str(string))
    if match:
        res = int(match.group())
        if 1980<=res <= 2016  : return res

    else: return None

In [ ]:
data.drop(columns= 'year_in_name', inplace=True)

In [76]:
data['year_in_name'] = data.name.apply(find_year_in_name_2)

In [77]:
data[~data.year_in_name.isna()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,year_in_name
87,Star Wars Battlefront (2015),PS4,2015.0,Shooter,2.99,3.49,0.22,1.28,NaN,NaN,NaN,2015.0
183,Madden NFL 2004,PS2,NaN,Sports,4.26,0.26,0.01,0.71,94.0,8.5,E,2004.0
238,Madden NFL 2005,PS2,2004.0,Sports,4.18,0.26,0.01,0.08,91.0,7.9,E,2005.0
250,Winning Eleven: Pro Evolution Soccer 2007,PS2,2006.0,Sports,0.10,2.39,1.05,0.86,86.0,8.9,E,2007.0
283,Madden NFL 2003,PS2,2002.0,Sports,3.36,0.21,0.01,0.56,95.0,7.7,E,2003.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16638,Tour de France 2014,PS3,2014.0,Sports,0.00,0.01,0.00,0.00,NaN,tbd,E,2014.0
16658,Driving Simulator 2011,PC,2011.0,Racing,0.00,0.01,0.00,0.00,NaN,NaN,NaN,2011.0
16676,G1 Jockey 4 2008,PS3,2008.0,Sports,0.00,0.00,0.00,0.00,NaN,NaN,NaN,2008.0
16711,LMA Manager 2007,X360,2006.0,Sports,0.00,0.01,0.00,0.00,NaN,NaN,NaN,2007.0


Выберем только те строки, у которых в обоих столбца по годам нет пустых значений

In [83]:
data[(~data.year_of_release.isna()) & (~data.year_in_name.isna())]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,year_in_name
87,Star Wars Battlefront (2015),PS4,2015.0,Shooter,2.99,3.49,0.22,1.28,NaN,NaN,NaN,2015.0
238,Madden NFL 2005,PS2,2004.0,Sports,4.18,0.26,0.01,0.08,91.0,7.9,E,2005.0
250,Winning Eleven: Pro Evolution Soccer 2007,PS2,2006.0,Sports,0.10,2.39,1.05,0.86,86.0,8.9,E,2007.0
283,Madden NFL 2003,PS2,2002.0,Sports,3.36,0.21,0.01,0.56,95.0,7.7,E,2003.0
333,Mario & Sonic at the London 2012 Olympic Games,Wii,2011.0,Sports,1.12,1.87,0.27,0.45,NaN,NaN,NaN,2012.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16638,Tour de France 2014,PS3,2014.0,Sports,0.00,0.01,0.00,0.00,NaN,tbd,E,2014.0
16658,Driving Simulator 2011,PC,2011.0,Racing,0.00,0.01,0.00,0.00,NaN,NaN,NaN,2011.0
16676,G1 Jockey 4 2008,PS3,2008.0,Sports,0.00,0.00,0.00,0.00,NaN,NaN,NaN,2008.0
16711,LMA Manager 2007,X360,2006.0,Sports,0.00,0.01,0.00,0.00,NaN,NaN,NaN,2007.0


Очень похоже на то, что игра выпущена либо в тот же год, либо на следующий

In [87]:
temp_df =  data[(~data.year_of_release.isna()) & (~data.year_in_name.isna())]\
            [['name', 'year_of_release', 'year_in_name']]

temp_df['check_year'] = abs(temp_df.year_of_release - temp_df.year_in_name) <=1
temp_df.head()

,name,year_of_release,year_in_name,check_year
87,Star Wars Battlefront (2015),2015.0,2015.0,True
238,Madden NFL 2005,2004.0,2005.0,True
250,Winning Eleven: Pro Evolution Soccer 2007,2006.0,2007.0,True
283,Madden NFL 2003,2002.0,2003.0,True
333,Mario & Sonic at the London 2012 Olympic Games,2011.0,2012.0,True


In [89]:
temp_df.shape

(815, 4)

In [88]:
temp_df.check_year.sum()

784

Так, получается, что 31 строка не следует логике, что игра выпускается либо в этот же год, что указан в названии, либо на следующий.

Посмотрим что это за значения

In [90]:
temp_df[~temp_df.check_year]

,name,year_of_release,year_in_name,check_year
769,SimCity 2000,1992.0,2000.0,False
926,FIFA 2000,1998.0,2000.0,False
1004,Madden NFL 2000,1998.0,2000.0,False
1237,NBA Live 2000,1998.0,2000.0,False
1646,SimCity 2000,1996.0,2000.0,False
1891,NFL Blitz 2000,1998.0,2000.0,False
2076,NFL Fever 2002,2000.0,2002.0,False
2395,NHL 2000,1998.0,2000.0,False
3042,Tiger Woods PGA Tour 2000,1998.0,2000.0,False
4066,SimCity 2000,1995.0,2000.0,False


Разброс может быть разным. 
посчитаем разницу в годах и построим распределение